In [21]:
# !pip3 install fairseq
# !pip3 install fastbpe
# !pip3 install transformers

In [22]:
# download và giải nén model từ package transformers
# wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
# tar -xzvf PhoBERT_base_transformers.tar.gz

# Import thư viện

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import time
import re
import random
import seaborn as sns
from tqdm.notebook import tqdm
import torch
import argparse
import pickle
import numpy as np
from os.path import join

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.autograd import Variable
from torch.backends import cudnn
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from sklearn.manifold import TSNE
from transformers.modeling_utils import * 
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
from transformers import AutoModel, AutoTokenizer

import argparse

In [3]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [25]:
# from fairseq.models.roberta import RobertaModel
# from fairseq.data.encoders.fastbpe import fastBPE
# from fairseq.data import Dictionary

# Load data

In [4]:
data = pd.read_excel("../dataset/final_data.1.xlsx")

In [5]:
data.isna().sum()

Rate                  0
Review                0
Label                 0
Preprocess_Review    30
dtype: int64

In [6]:
data.dropna(axis=0, inplace=True)

In [7]:
reviews = data["Preprocess_Review"]
labels = data["Label"]

## Chia tập train, test

In [8]:
train_x, test_x, train_y, test_y = train_test_split(reviews, labels, test_size=0.2, random_state=42, stratify=labels)


In [9]:
set(labels)

{-1, 0, 1}

In [10]:
set(train_y)

{-1, 0, 1}

# Sử dụng PhoBERT là feature extractor

## 1. Load model pretrained PhoBERT

In [11]:
# load model phoBert và tokenizer của model đó
phoBert = AutoModel.from_pretrained('vinai/phobert-base')
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 2. Tokenize văn bản và đưa về một kích thước cố định

In [12]:
# Đưa từng sentence qua tokenizer của PhoBERT để convert sang dạng token index với cùng chiều dài
# params
MAX_SEQ_LEN = 256 # chiều dài tối đa của một câu
# id của 1 số token đặc biệt
cls_id = 0  # đầu câu
eos_id = 2  # cuối câu
pad_id = 1  # padding

# Hàm xử lý dữ liệu trên từng sentence
def tokenize_line(line):
  tokenized = tokenizer.encode(line)
  
  l = len(tokenized)
  if l > MAX_SEQ_LEN: # nếu dài hơn thì cắt bỏ
    tokenized = tokenized[:MAX_SEQ_LEN]
    tokenized[-1] = eos_id # thêm EOS vào cuối câu
  else: # nếu ngắn hơn thì thêm padding vào
    tokenized = tokenized + [pad_id, ] * (MAX_SEQ_LEN - l)
  
  return tokenized

In [13]:

# Ví dụ
tokenized = tokenize_line('tôi khỏe')
print('After encode: ', tokenized)
print('Token size: ', len(tokenized))

After encode:  [0, 70, 3014, 15947, 1565, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Token size:  256


In [14]:
train_x.isna().sum()

0

In [15]:

# Xử lý trên toàn tập dữ liệu
tokenized_train_x = []
tokenized_test_x = []

start = time.time()

print('Tokenizing train set ...')
for x in tqdm(train_x):
  tokenized_train_x.append(tokenize_line(x))

print('Tokenizing test set ... ')
for x in tqdm(test_x):
  tokenized_test_x.append(tokenize_line(x))

print('Done after %d seconds' %(time.time() - start))

Tokenizing train set ...


  0%|          | 0/9160 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (263 > 256). Running this sequence through the model will result in indexing errors


Tokenizing test set ... 


  0%|          | 0/2291 [00:00<?, ?it/s]

Done after 2 seconds


In [16]:

# Tạo attention mask để model chỉ tập trung vào phần nội dung mà không quan tâm tới phần padding
train_attention_mask = []
for x in tokenized_train_x:
  train_attention_mask.append(np.where(np.array(x) == 1, 0, 1))

test_attention_mask = []
for x in tokenized_test_x:
  test_attention_mask.append(np.where(np.array(x) == 1, 0, 1))

print('Done!')

Done!


In [17]:
print(train_attention_mask[10], len(train_attention_mask[10]))

[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 256


## 3. Đưa dữ liệu kèm theo attention mash vào model PhoBERT

In [18]:

# convert sang tensor
tokenized = torch.tensor(tokenized_train_x[:1]).to(torch.long)
attention_mask = torch.tensor(train_attention_mask[:1])

with torch.no_grad():
  last_hidden_states = phoBert(input_ids=tokenized, attention_mask=attention_mask)

print(last_hidden_states[0].shape)

C:\Users\anhdu\AppData\Local\Temp\ipykernel_18272\4075655186.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  attention_mask = torch.tensor(train_attention_mask[:1])


torch.Size([1, 256, 768])


***Nhận xét:*** Dữ liệu trả về là một tensor có size là (1,256,768) trong đó 1 là số sentence truyền vào, 256 là độ dài mỗi sentence và 768 là vector embedding cho mỗi word của sentence đó. Bởi vì ta sẽ sử dụng vector embedding của token CLS để classify nên ta sẽ lấy vector embedding của từ đầu tiên của mọi câu.

In [ ]:
# Vector embedding của train set
tokenized = torch.tensor(tokenized_train_x[:]).to(torch.long)
attention_mask = torch.tensor(train_attention_mask[:])

start = time.time()

with torch.no_grad():
  last_hidden_states = phoBert(input_ids=tokenized, attention_mask=attention_mask)

train_features = last_hidden_states[0][:, 0, :].numpy()
print('Finish extracting features after %d seconds' %(time.time() - start))

print('Output shape: ', train_features.shape)

In [20]:
# Hàm extract feature của 1 line
def extract_line(tokenized, mask):
  tokenized = torch.tensor(tokenized).to(torch.long)
  mask = torch.tensor(mask)

  with torch.no_grad():
    last_hidden_states = phoBert(input_ids=tokenized, attention_mask=mask)
  
  feature = last_hidden_states[0][:, 0, :].numpy()

  return feature

In [28]:
document = "k ngon"

tokenized = [tokenize_line(document)]

mask = [np.where(np.array(tokenized) == 1, 0, 1)]

extract_line(tokenized, mask)

array([[ 1.45886883e-01,  4.27036107e-01, -2.68668413e-01,
        -6.65369391e-01, -9.71091688e-02, -3.94507557e-01,
         3.59896243e-01, -3.44905198e-01, -2.16611221e-01,
         2.18474507e-01, -1.73639789e-01, -3.02961111e-01,
        -3.46201658e-03,  4.33996320e-01,  6.83624685e-01,
         8.70250165e-02, -2.64169753e-01, -2.88108408e-01,
         2.90218741e-03, -4.64014769e-01,  4.41070974e-01,
        -1.27252877e+00,  6.80580616e-01, -2.86518931e-02,
         1.73290893e-01,  3.42010677e-01,  1.10351369e-01,
        -8.96850228e-03, -2.76633084e-01,  4.97711331e-01,
        -2.02434570e-01, -2.44635433e-01,  1.85743377e-01,
         1.86224848e-01,  3.79489899e-01,  5.96038364e-02,
        -4.72787395e-02, -2.38899916e-01,  1.22735277e-01,
         6.64257169e-01,  4.95660126e-01,  1.96896791e-02,
         1.62627295e-01, -6.90609515e-01, -1.83929831e-01,
        -2.72066861e-01,  3.42366219e-01,  3.81594062e-01,
        -1.00714713e-01,  4.04931009e-01, -1.15194589e-0

In [22]:
i = 3
step = 1
mask = test_attention_mask[i:i+step]
tokenized = tokenized_test_x[i:i+step]

In [27]:
mask.shape

AttributeError: 'list' object has no attribute 'shape'

In [ ]:
# Embed tập train
train_size = len(tokenized_train_x)
EMBED_SIZE = 768
train_features = np.zeros(shape=(train_size, EMBED_SIZE))

start = time.time()

step = 20
for i in tqdm(range(0, train_size, step)):
  mask = train_attention_mask[i:i+step]
  tokenized = tokenized_train_x[i:i+step]
  feature = extract_line(tokenized, mask)
  train_features[i:i+step] = feature

print('Finish extracting features after %d seconds' %(time.time() - start))
print('Output shape: ', train_features.shape)

In [59]:
# Thực hiện tương tự trên tập test
test_size = len(tokenized_test_x)
EMBED_SIZE = 768
test_features = np.zeros(shape=(test_size, EMBED_SIZE))

start = time.time()

step = 20
for i in tqdm(range(0, test_size, step)):
  mask = test_attention_mask[i:i+step]
  tokenized = tokenized_test_x[i:i+step]
  feature = extract_line(tokenized, mask)
  test_features[i:i+step] = feature

print('Finish extracting features after %d seconds' %(time.time() - start))
print('Output shape: ', test_features.shape)

  0%|          | 0/115 [00:00<?, ?it/s]

Finish extracting features after 909 seconds
Output shape:  (2291, 768)


In [60]:
import os
import pickle

# xác định đường dẫn tới thư mục hiện tại
current_dir = os.path.dirname(os.path.abspath("4. phoBERT.ipynb"))

# đường dẫn tới file cần lưu
bert_features_path = os.path.join(current_dir, 'bert_features.sav')



In [61]:
# lưu danh sách embed
pickle.dump(train_features, open(bert_features_path, 'wb'))

In [62]:
bert_test_features_path = os.path.join(current_dir, 'bert_test_features.sav')

In [63]:
pickle.dump(test_features, open(bert_test_features_path, 'wb'))


Đánh giá khả năng phân cụm của PhoBERT

In [64]:

# visualize embedded vector dưới góc nhìn 2 chiều
tsne = TSNE(n_components=2, random_state=0)
x_2d = tsne.fit_transform(train_features)
print('Embedded shape: ', x_2d.shape)

Embedded shape:  (9160, 2)


In [65]:
# # visualize
# plt.figure(figsize=(20, 12))
# color_map = {
#     0: 'red', 1: 'blue', 2: 'lightgreen'
# }
# # labels = ["negative", "neutral", "positive"]

# train_y = np.array(train_y)

# for idx, label in enumerate(labels):
#   plt.scatter(x=x_2d[train_y == label, 0], y=x_2d[train_y == label, 1], c=color_map[idx], marker='o', label=label)

# plt.xlabel('X')
# plt.ylabel('Y')
# plt.legend()
# plt.title('2D Visualization of Training Data')
# plt.show()

# Train classifier

In [66]:
# load vector embedding đã lưu
train_features = pickle.load(open(bert_features_path, 'rb'))
test_features = pickle.load(open(bert_test_features_path, 'rb'))

## Logistic regression

In [67]:
logisticReg = LogisticRegression(max_iter=100_000)

start = time.time()
logisticReg.fit(train_features, train_y)

print('Logistic Regression finish training after %d second' %(time.time() - start))

bert_logreg_path = './bert_logreg.sav'
pickle.dump(logisticReg, open(bert_logreg_path, 'wb'))

Logistic Regression finish training after 9 second


In [68]:
# Đánh giá độ chính xác của mô hình Logistic Regression
y_pred = logisticReg.predict(test_features)

acc = np.mean(y_pred == np.array(test_y))

print('Accuracy: ', acc)

Accuracy:  0.7642950676560454


In [69]:

print(classification_report(test_y, y_pred, zero_division=1))

              precision    recall  f1-score   support

          -1       0.71      0.73      0.72       675
           0       0.29      0.14      0.19       239
           1       0.82      0.89      0.85      1377

    accuracy                           0.76      2291
   macro avg       0.61      0.59      0.59      2291
weighted avg       0.74      0.76      0.75      2291



In [70]:
# visualize bằng heat map
plt.figure(figsize=(16, 16))
conf_matrix = confusion_matrix(test_y, y_pred, labels=labels)
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='YlOrRd' , xticklabels=labels, yticklabels=labels)
     

## SVM

In [ ]:

svm = SVC()

start = time.time()
svm.fit(train_features, train_y)
print('SVM classifier finish training after %d seconds' %(time.time() - start))

bert_svm_path = './bert_svm.sav'
pickle.dump(svm, open(bert_svm_path, 'wb'))

SVM classifier finish training after 23 seconds


In [ ]:

# Đánh giá độ chính xác
y_pred = svm.predict(test_features)

acc = np.mean(y_pred == np.array(test_y))
print('Accuracy: ', acc)

Accuracy:  0.7861195984286338


In [ ]:
print(classification_report(test_y, y_pred, zero_division=1))


              precision    recall  f1-score   support

          -1       0.74      0.76      0.75       675
           0       0.50      0.00      0.01       239
           1       0.81      0.93      0.87      1377

    accuracy                           0.79      2291
   macro avg       0.68      0.57      0.54      2291
weighted avg       0.75      0.79      0.74      2291



In [ ]:
# visualize bằng heat map
plt.figure(figsize=(16, 16))
conf_matrix = confusion_matrix(test_y, y_pred, labels=labels)
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='YlOrRd' , xticklabels=labels, yticklabels=labels)
     

In [ ]:
import joblib

joblib.dump(svm, "clf_svm.pkl")

['clf_svm.pkl']